In [1]:
#import matplotlib as mpl
#from mpl_toolkits.mplot3d import Axes3D
import numpy as np
#import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#import quadpy
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations
from joblib import Parallel, delayed, parallel_backend
from scipy import integrate
import time
from datetime import timedelta
import plotly as plotly
import plotly.express as px
#from plotly.offline import init_notebook_mode
#init_notebook_mode(connected=True)  
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.graph_objects as go

import os

def crystal(i):
    switcher={
        45: "Li", 
        44: "Cu",
        43: "Mn",
        42: "Al",            
        41: "Au",            
        40: "Ca",      
        39: "Cr",        
        38: "Fe",            
        37: "Ge",            
        36: "Ir",      
        35: "Pd",        
        34: "Pt",            
        33: "Ru",            
        32: "Si",      
        31: "Tc",
        30: "Ti",
        29: "V",
        28: "W",
        27: "Mg",
        26: "Co",
        25: "Sc", 
        24: "Hg",
        23: "Mo", 
        22: "Os",
        21: "Rh",
        20: "Ag",
        19: "Cd",
        18: "Hf",
        17: "K",
        16: "Na",
        15: "Nb",
        14: "Ni",
        13: "Re",
        12: "Sb",
        11: "Sr",
        10: "Ta",
        9: "Y",
        8: "Zr",
        7: "Ba",
        6: "Cs",
        5: "In",
        4: "Rb",
        3: "Sn",
        2: "Pb",
        1: "Tl",
    }
    return switcher.get(i,"Invalid crystal")

import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)  
import plotly.io as pio
pio.renderers.default = 'iframe'

df_TETRA = pd.read_csv("NotIntCharge_TETRA.csv")
# df_TETRA = pd.read_csv("NotIntCharge_TETRA.csv")
R_cont=[]
for nx in range (0,101):
    for ny in range(0,101):
        for nz in range(0,101):
            
            R_cont.append(np.sqrt(nx**2+ny**2+nz**2))

R_cont=np.array(R_cont)

R=R_cont.astype(int)

Rr=np.unique(R_cont.ravel()) 

Rrmax=max(Rr)
nr = np.bincount(R)
Rr_axis=np.linspace(0,Rrmax,len(nr)) #== Rr?
print(Rr_axis)
suffix = "TETRA"

fig = go.Figure()
data_OI=[]

axis_template_x=dict(
    title="H",
    linecolor='black',
    linewidth=2,
    # tickfont = dict(size=0),
    titlefont = dict(size = 33),
    showticklabels=False
)
axis_template_y=dict(
    title="RDF",
    linecolor='black',
    linewidth=2,
    tickfont = dict(size=33),
    titlefont = dict(size = 33)
)
layout=go.Layout(
xaxis=axis_template_x,
yaxis=axis_template_y,
#paper_bgcolor='rgba(0,0,0,0)',
plot_bgcolor='rgba(0,0,0,0)',
width=700, height=1000
)
cc=1
actual_max=[]
for pp in range (1,46):
    
    if pp==7 or pp==26 or pp==38 or pp==14 or pp==28 or pp==18 or pp==24:
        continue
        
    if pp==6 or pp==32 or pp==12:
        continue
        
    crystalname=crystal(pp)
    columnname = crystalname+'_'+suffix+'_RDF'
    RDF = df_TETRA[columnname]
    
    print(len(RDF))

    RDF = np.array(RDF)

    RDF = RDF[np.logical_not(np.isnan(RDF))]
    
    RDF_edit=np.roll(RDF, 50)

    charge_DF={"R":Rr_axis,
               "RDF (TI)":RDF_edit,
              "crystal":crystalname}
    
    #fig.add_trace(
    # if pp==16 or pp==17 or pp==11 or pp==7 or pp==9 or pp==25 or pp==5 or pp==1:
    # trace=go.Scatter(x=charge_DF['R'], y=charge_DF['RDF'] , mode='lines', textfont=dict(
    #         size=15))
    #else: 
    trace=go.Scatter(x=charge_DF['R'], y=charge_DF['RDF (TI)'] , name=charge_DF['crystal'], text=crystalname, mode='lines', textfont=dict(
           size=15))

    
    actual_max.append(max(RDF_edit))
    # print(actual_max)
    # print(cc)
    # print(actual_max[cc-1]>actual_max[cc-2])
    inc=0
    if cc>=2:
        if actual_max[cc-1]>actual_max[cc-2]:
            inc=3
        else:
            inc=-3

    trace1=go.Scatter(showlegend=False,x=[70+inc],y=[max(RDF_edit)],mode='text',text=crystalname,textfont=dict(size=19))
    trace2=go.Scatter(
        x = [60.06, 67+inc],
        y = [max(RDF_edit), max(RDF_edit)],
        mode = "lines",
        marker = dict(color = 'black'))
    
    cc+=1
    #)
    #fig.update_traces(textposition='top center')
    #fig.update_xaxes(rangebreaks=list(name='x'))
    data_OI.append(trace)
    data_OI.append(trace1)
    data_OI.append(trace2)

fig=go.Figure(data=data_OI, layout=layout)

#plotly.offline.plot(fig, filename='/Users/dariomassa/Documents/PHD_NCBJ/FCC_new/SPIN_CHECK/RDF_All_'+suffix+'.html')

# Only thing I figured is - I could do this 
#fig.add_scatter(x=df['Date'], y=df['AAPL.Low']) # Not what is desired - need a line 

# fig.add_trace(go.Scatter(x=[12.05],y=[0.11],mode='text',text='Cr',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[15.05],y=[0.096],mode='text',text='Cu',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[18.05],y=[0.081],mode='text',text='V',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[20.05],y=[0.063],mode='text',text='Ti',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[22.05],y=[0.046],mode='text',text='Sc',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[24.05],y=[0.025],mode='text',text='Na',textfont=dict(size=33)))

# fig.add_trace(go.Scatter(x=[14.05],y=[0.042],mode='text',text='K',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[27.11],y=[0.005],mode='text',text='In',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[27.11],y=[0.000],mode='text',text='Tl',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[14],y=[0.0148],mode='text',text='Sr',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[16],y=[0.010],mode='text',text='Ba',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[14.05],y=[0.030],mode='text',text='Sc',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[14.05],y=[0.026],mode='text',text='Y',textfont=dict(size=21)))

# Show plot 
fig.show()
plotly.offline.plot(fig, filename='./RDF_All_'+suffix+'.html')
#fig1.show()



[  0.           1.00118544   2.00237088   3.00355631   4.00474175
   5.00592719   6.00711263   7.00829807   8.0094835    9.01066894
  10.01185438  11.01303982  12.01422525  13.01541069  14.01659613
  15.01778157  16.01896701  17.02015244  18.02133788  19.02252332
  20.02370876  21.0248942   22.02607963  23.02726507  24.02845051
  25.02963595  26.03082139  27.03200682  28.03319226  29.0343777
  30.03556314  31.03674857  32.03793401  33.03911945  34.04030489
  35.04149033  36.04267576  37.0438612   38.04504664  39.04623208
  40.04741752  41.04860295  42.04978839  43.05097383  44.05215927
  45.05334471  46.05453014  47.05571558  48.05690102  49.05808646
  50.0592719   51.06045733  52.06164277  53.06282821  54.06401365
  55.06519908  56.06638452  57.06756996  58.0687554   59.06994084
  60.07112627  61.07231171  62.07349715  63.07468259  64.07586803
  65.07705346  66.0782389   67.07942434  68.08060978  69.08179522
  70.08298065  71.08416609  72.08535153  73.08653697  74.0877224
  75.0889078

'./RDF_All_TETRA.html'

In [2]:
#import matplotlib as mpl
#from mpl_toolkits.mplot3d import Axes3D
import numpy as np
#import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#import quadpy
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations
from joblib import Parallel, delayed, parallel_backend
from scipy import integrate
import time
from datetime import timedelta
import plotly as plotly
import plotly.express as px
#from plotly.offline import init_notebook_mode
#init_notebook_mode(connected=True)  
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.graph_objects as go

import os

def crystal(i):
    switcher={
        45: "Li", 
        44: "Cu",
        43: "Mn",
        42: "Al",            
        41: "Au",            
        40: "Ca",      
        39: "Cr",        
        38: "Fe",            
        37: "Ge",            
        36: "Ir",      
        35: "Pd",        
        34: "Pt",            
        33: "Ru",            
        32: "Si",      
        31: "Tc",
        30: "Ti",
        29: "V",
        28: "W",
        27: "Mg",
        26: "Co",
        25: "Sc", 
        24: "Hg",
        23: "Mo", 
        22: "Os",
        21: "Rh",
        20: "Ag",
        19: "Cd",
        18: "Hf",
        17: "K",
        16: "Na",
        15: "Nb",
        14: "Ni",
        13: "Re",
        12: "Sb",
        11: "Sr",
        10: "Ta",
        9: "Y",
        8: "Zr",
        7: "Ba",
        6: "Cs",
        5: "In",
        4: "Rb",
        3: "Sn",
        2: "Pb",
        1: "Tl",
    }
    return switcher.get(i,"Invalid crystal")

import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)  
import plotly.io as pio
pio.renderers.default = 'iframe'

df_OCTA = pd.read_csv("NotIntCharge_OCTA.csv")
# df_OCTA = pd.read_csv("NotIntCharge_OCTA.csv")
R_cont=[]
for nx in range (0,101):
    for ny in range(0,101):
        for nz in range(0,101):
            
            R_cont.append(np.sqrt(nx**2+ny**2+nz**2))

R_cont=np.array(R_cont)

R=R_cont.astype(int)

Rr=np.unique(R_cont.ravel()) 

Rrmax=max(Rr)
nr = np.bincount(R)
Rr_axis=np.linspace(0,Rrmax,len(nr)) #== Rr?

suffix = "OCTA"

fig = go.Figure()
data_OI=[]

axis_template_x=dict(
    title="H",
    linecolor='black',
    linewidth=2,
    # tickfont = dict(size=0),
    titlefont = dict(size = 33),
    showticklabels=False
)
axis_template_y=dict(
    title="",
    linecolor='black',
    linewidth=2,
    tickfont = dict(size=33),
    titlefont = dict(size = 33)
)
layout=go.Layout(
xaxis=axis_template_x,
yaxis=axis_template_y,
#paper_bgcolor='rgba(0,0,0,0)',
plot_bgcolor='rgba(0,0,0,0)',
width=700, height=1000
)
cc=1
actual_max=[]
for pp in range (1,46):
    
    if pp==7 or pp==26 or pp==38 or pp==14 or pp==28 or pp==18 or pp==24:
        continue
        
    if pp==6 or pp==32 or pp==12:
        continue
        
    crystalname=crystal(pp)
    columnname = crystalname+'_'+suffix+'_RDF'
    RDF = df_OCTA[columnname]

    RDF = np.array(RDF)

    RDF = RDF[np.logical_not(np.isnan(RDF))]
    
    RDF_edit=np.roll(RDF, 50)

    charge_DF={"R":Rr_axis,
               "RDF (OI)":RDF_edit,
              "crystal":crystalname}
    
    #fig.add_trace(
    # if pp==16 or pp==17 or pp==11 or pp==7 or pp==9 or pp==25 or pp==5 or pp==1:
    # trace=go.Scatter(x=charge_DF['R'], y=charge_DF['RDF'] , mode='lines', textfont=dict(
    #         size=15))
    #else: 
    trace=go.Scatter(x=charge_DF['R'], y=charge_DF['RDF (OI)'] , name=charge_DF['crystal'], text=crystalname, mode='lines', textfont=dict(
           size=15))

    
    actual_max.append(max(RDF_edit))
    # print(actual_max)
    # print(cc)
    # print(actual_max[cc-1]>actual_max[cc-2])
    inc=0
    if cc>=2:
        if actual_max[cc-1]>actual_max[cc-2]:
            inc=3
        else:
            inc=-3

    trace1=go.Scatter(showlegend=False,x=[70+inc],y=[max(RDF_edit)],mode='text',text=crystalname,textfont=dict(size=19))
    trace2=go.Scatter(
        x = [60.06, 67+inc],
        y = [max(RDF_edit), max(RDF_edit)],
        mode = "lines",
        marker = dict(color = 'black'))
    
    cc+=1
    #)
    #fig.update_traces(textposition='top center')
    #fig.update_xaxes(rangebreaks=list(name='x'))
    data_OI.append(trace)
    data_OI.append(trace1)
    data_OI.append(trace2)

fig=go.Figure(data=data_OI, layout=layout)

#plotly.offline.plot(fig, filename='/Users/dariomassa/Documents/PHD_NCBJ/FCC_new/SPIN_CHECK/RDF_All_'+suffix+'.html')

# Only thing I figured is - I could do this 
#fig.add_scatter(x=df['Date'], y=df['AAPL.Low']) # Not what is desired - need a line 

# fig.add_trace(go.Scatter(x=[12.05],y=[0.11],mode='text',text='Cr',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[15.05],y=[0.096],mode='text',text='Cu',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[18.05],y=[0.081],mode='text',text='V',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[20.05],y=[0.063],mode='text',text='Ti',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[22.05],y=[0.046],mode='text',text='Sc',textfont=dict(size=33)))
# fig.add_trace(go.Scatter(x=[24.05],y=[0.025],mode='text',text='Na',textfont=dict(size=33)))

# fig.add_trace(go.Scatter(x=[14.05],y=[0.042],mode='text',text='K',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[27.11],y=[0.005],mode='text',text='In',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[27.11],y=[0.000],mode='text',text='Tl',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[14],y=[0.0148],mode='text',text='Sr',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[16],y=[0.010],mode='text',text='Ba',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[14.05],y=[0.030],mode='text',text='Sc',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[14.05],y=[0.026],mode='text',text='Y',textfont=dict(size=21)))

# Show plot 
fig.show()
plotly.offline.plot(fig, filename='./RDF_All_'+suffix+'.html')
#fig1.show()



'./RDF_All_OCTA.html'

In [39]:
suffix = "OCTA"

fig = go.Figure()
data=[]

axis_template_x=dict(
    title="R",
    linecolor='black',
    linewidth=2,
    tickfont = dict(size=20),
    titlefont = dict(size = 25)
)
axis_template_y=dict(
    title="RDF",
    linecolor='black',
    linewidth=2,
    tickfont = dict(size=20),
    titlefont = dict(size = 25)
)
layout=go.Layout(
xaxis=axis_template_x,
yaxis=axis_template_y,
#paper_bgcolor='rgba(0,0,0,0)',
plot_bgcolor='rgba(0,0,0,0)',
width=800, height=500
)
for pp in range (43,44):
    
    if pp==7 or pp==26 or pp==38 or pp==14 or pp==28 or pp==18 or pp==24:
        continue
        
    if pp==6 or pp==32 or pp==12:
        continue
        
    crystalname=crystal(pp)
    columnname = crystalname+'_'+suffix+'_RDF'
    RDF = df_OCTA[columnname]

    RDF = np.array(RDF)

    RDF = RDF[np.logical_not(np.isnan(RDF))]

    charge_DF={"R":Rr_axis,
               "RDF":RDF,
              "crystal":crystalname}
    
    #fig.add_trace(
    # if pp==16 or pp==17 or pp==11 or pp==7 or pp==9 or pp==25 or pp==5 or pp==1:
    # trace=go.Scatter(x=charge_DF['R'], y=charge_DF['RDF'] , mode='lines', textfont=dict(
    #         size=15))
    #else: 
    trace=go.Scatter(showlegend=False,x=charge_DF['R'], y=charge_DF['RDF'] , name=charge_DF['crystal'], text=crystalname, mode='markers', textfont=dict(
           size=34),marker=dict(size=10,color=charge_DF['RDF'],colorscale='Rainbow',showscale=True,colorbar=dict(thickness=20,
                           tickfont=dict(size=34)))
                    )
    #)
    #fig.update_traces(textposition='top center')
    #fig.update_xaxes(rangebreaks=list(name='x'))
    data.append(trace)
    
fig=go.Figure(data=data, layout=layout)

#plotly.offline.plot(fig, filename='/Users/dariomassa/Documents/PHD_NCBJ/FCC_new/SPIN_CHECK/RDF_All_'+suffix+'.html')

# Only thing I figured is - I could do this 
#fig.add_scatter(x=df['Date'], y=df['AAPL.Low']) # Not what is desired - need a line 

# fig.add_trace(go.Scatter(x=[14.05],y=[0.049],mode='text',text='Na',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[14.05],y=[0.042],mode='text',text='K',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[27.11],y=[0.005],mode='text',text='In',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[27.11],y=[0.000],mode='text',text='Tl',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[14],y=[0.0148],mode='text',text='Sr',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[16],y=[0.010],mode='text',text='Ba',textfont=dict(size=21)))

# fig.add_trace(go.Scatter(x=[14.05],y=[0.030],mode='text',text='Sc',textfont=dict(size=21)))
# fig.add_trace(go.Scatter(x=[14.05],y=[0.026],mode='text',text='Y',textfont=dict(size=21)))


fig.add_trace(go.Scatter(showlegend=False,x=[80],y=[0.05],mode='text',text='Mn(OI)',textfont=dict(size=34)))
fig.update_layout(font={"size":34})

axis_template_x=dict(
    linecolor='black',
    linewidth=2,
    title="R",
    tickfont = dict(size=34),
    titlefont = dict(size=34)
)
axis_template_y=dict(
    linecolor='black',
    linewidth=2,
    tickfont = dict(size=34),
    titlefont = dict(size=34),
    title="RDF"
)
fig.add_vline(x=0.0,line_width=1, line_dash="dash", line_color="black", opacity=0.5)
fig.add_hline(y=0.0,line_width=1, line_dash="dash", line_color="black", opacity=0.5)
# Show plot 

fig.update_xaxes(

        tickfont = {"size": 34},
        title_font = {"size": 34},)

fig.update_yaxes(
        tickfont = {"size": 34},
        title_font = {"size": 34},)

fig.show()
#fig1.show()
plotly.offline.plot(fig, filename='./RDF_Mn_Example_'+suffix+'.html')


'./RDF_Mn_Example_OCTA.html'

In [40]:
len(RDF_edit)

174